In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

import seaborn as sns

In [2]:
#First let's clean the dataset

In [3]:
df = pd.read_csv('../../data/train.csv')

In [4]:
df.head()

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,text
0,0,1588696955143,0,False,68460,1101,1226,NaN,NaN,NaN,Smh I give up
1,1,1588464948124,0,False,309,51,202,NaN,NaN,NaN,"Most of us are Human Beings, but I think you m..."
2,2,1588634673360,0,False,3241,1675,2325,NaN,NaN,NaN,"Old dirty tricks Trump, at it again...like we ..."
3,3,1588433158672,0,False,32327,667,304,NaN,NaN,NaN,Seriously..... I worked 86 hours my last check...
4,4,1588582751599,0,False,581,42,127,NaN,NaN,NaN,May ALMIGHTY ALLAH have mercy on us all. Only ...


### Function used to add the hashtags from text to 'hashtags' column

In [5]:
def clean_hashtag(row):
    return " ".join(filter(lambda x:x[0]!='#', row.split()))

def delete_duplicates(string):
    words = string.split(',')
    words = list(map(str.strip, words))
    return " ".join(sorted(set(words), key=words.index))

In [6]:
#Remove hashtags from text
def find_deplace_hashtags(df):
    '''
    This function moves the "#" from the text directly to the 'hashtags' columns.
    '''
    #Find extra hashtags
    extra_hashtags = df.apply(lambda x: re.findall("[#]\w+", x.text), axis=1)
    
    clean_hashtags = []
    for ls in extra_hashtags:
        res = ', '.join(ls)
        res = res.replace('#','')
        clean_hashtags.append(res)
    
    print(clean_hashtags)
    df.text = df.apply(lambda x: clean_hashtag(x.text), axis=1)
    
    #add them to hashtags columns
    df['hashtags'] = df['hashtags'] + ', ' + pd.Series(clean_hashtags)
    df.hashtags = df.apply(lambda x : delete_duplicates(str(x.hashtags)), axis=1)

### Before

In [7]:
print(df.iloc[665773].hashtags)
print(df.iloc[665773].text)
print(df.iloc[665773].urls)

COVID19
Thank you to all of the nurses in our @StanfordEMED family sharing the #COVID19 frontlines. We are definitely #strongertogether because of you! https://t.co/X1XaJjhbsO
twitter.com/i/web/status/1…


### After

In [8]:
find_deplace_hashtags(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
print(df.iloc[665773].hashtags)
print(df.iloc[665773].text)

COVID19 strongertogether
Thank you to all of the nurses in our @StanfordEMED family sharing the frontlines. We are definitely because of you! https://t.co/X1XaJjhbsO


### Cleaning the URLS

In [10]:
def clean_urls(row):
    return " ".join(filter(lambda x:x[0:5]!='https', row.split()))

In [11]:
#Remove URLS
def find_deplace_urls(df):
    '''
    This function moves the "#" from the text directly to the 'hashtags' columns.
    '''
    #Find extra hashtags
    extra_urls = df.apply(lambda x: re.findall(r"https.*", x.text), axis=1)
    
    clean_urls_res = []
    for ls in extra_urls:
        res = ', '.join(ls)
        res = res.replace('nan, ', '')
        clean_urls_res.append(res)
   
    df.text = df.apply(lambda x: clean_urls(x.text), axis=1)
        
    #add them to urls columns  
    df.urls = df.apply(lambda x : delete_duplicates(str(x.urls)), axis=1)
    df["urls"] = df["urls"].str.cat(clean_urls_res, sep=', ')
    df.urls = df.urls.apply(lambda x: x.replace('nan, ', ''))
    df.urls = df.urls.replace('nan, ',np.NaN)

In [12]:
find_deplace_urls(df)

### After

In [13]:
print(df.iloc[665773].text)

Thank you to all of the nurses in our @StanfordEMED family sharing the frontlines. We are definitely because of you!


### Cleaning the user_mentions

In [14]:
def clean_at(row):
    return " ".join(filter(lambda x:x[0]!='@', row.split()))

In [15]:
#Remove hashtags from text
def find_deplace_ats(df):
    '''
    This function moves the "@" from the text directly to the 'hashtags' columns.
    '''
    #Find extra hashtags
    extra_hashtags = df.apply(lambda x: re.findall("[@]\w+", x.text), axis=1)
    
    clean_ats = []
    for ls in extra_hashtags:
        res = ', '.join(ls)
        res = res.replace('@','')
        clean_ats.append(res)
    
    df.text = df.apply(lambda x: clean_at(x.text), axis=1)
    
    #add them to hashtags columns
    df['user_mentions'] = df['user_mentions'] + ', ' + pd.Series(clean_ats)
    df.user_mentions = df.apply(lambda x : delete_duplicates(str(x.user_mentions)), axis=1)

In [16]:
find_deplace_ats(df)

### After #hashtags, @user_mentions and https links, we have only sentences in the text:

In [17]:
print(df.iloc[665773].hashtags)
print(df.iloc[665774].text)

COVID19 strongertogether
'Post it' pearls for Palliative, End of Life and Bereavement care in COVID-19 up the ED. Relating to symptom management, considerations for before and after death, honest conversations and goal setting


In [18]:
#Saving new csv
df.to_csv('../../data/train_cleanV1.csv')

### After

In [1]:
print(df.iloc[665773].text)

NameError: name 'df' is not defined

### Dealing with timestamps

In [ ]:
#from datetime import datetime
#df_2['timestamp'] = df['timestamp'].apply(lambda x: datetime.utcfromtimestamp(int(str(x))/1000).strftime("%Y-%m-%d %H:%M:%S"))

In [38]:
#Dealing with timestamps
import pandas as pd
df = pd.read_csv('../../data/train_cleanV4.csv')
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df.timestamp = pd.to_datetime(df.timestamp)

In [40]:
df['hour'] = df.timestamp.dt.hour
df['date'] = df.timestamp.dt.dayofweek

In [43]:
df.to_csv('../../data/train_cleanV4.csv')